In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
plt.rcdefaults()



/Users/seanm/opt/anaconda3/envs/SummerEnv/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
submission_dict = {}
fighters = []

Much credit to CGRetriever on Github, who I copied most of this webscraper from. 

https://github.com/CGRetriever/BJJ_Submission_Analysis/blob/master/BJJSubAnalyzer.py



In [ ]:
def submission_counter(submission, increment):
    # Making sure the increment variable is an int
    increment = int(increment)

    # This will attempt to increment the counter for the submission. If the submission does not exist then it will
    # add it to the dictionary instead (via the except loop)
    try:
        submission_dictionary[submission] += int(increment)
    except KeyError:
        submission_dictionary[submission] = int(increment)

In [ ]:
def get_fighters():
    # This will be grab the information from the bjjheroes website so that we can parse it
    page = requests.get(
        "https://www.bjjheroes.com/a-z-bjj-fighters-list")
    fighter_soup = BeautifulSoup(page.content, 'html.parser')

    # Search through the html for the first and last name of the fighters
    fighters_first_name = fighter_soup.find_all("td", class_='column-1')
    fighters_last_name = fighter_soup.find_all("td", class_='column-2')

    # Weird ass regex stuff. I don't understand it, I just copy and pasted this
    marker1 = ">"
    marker2 = "<"
    regexExpression = marker1 + '(.+?)' + marker2

    # This for loop is to create a full name for each fighter so that we can query the URLs with
    # each fighter's name
    for i, x in zip(fighters_first_name, fighters_last_name):
        tempFirst = str(i)
        tempLast = str(x)

        firstString = re.search(regexExpression, tempFirst).group(1)
        firstString = firstString.split(">")

        lastString = re.search(regexExpression, tempLast).group(1)
        lastString = lastString.split(">")
        fullname = firstString[1] + "-" + lastString[1]
        print(fullname)
        fighter_roster.append(fullname)
        web_scrape(fullname)


In [ ]:
def web_scrape(fighter):
    # This sets up the intial request to the site that we want to pull data from
    page = requests.get(
        "https://www.bjjheroes.com/bjj-fighters/" + fighter)
    soup = BeautifulSoup(page.content, 'html.parser')

    # print(soup)

    submissions_html = soup.find_all("div", class_='text')
    submission_numbers = soup.find_all("div", class_='totalvalue')
    example = soup.find_all(id="donut_value")
    marker1 = ">"
    marker2 = "<"
    regexExpression = marker1 + '(.+?)' + marker2

    for i, x in zip(submissions_html, submission_numbers):
        tempSubStr = str(i)
        tempNumStr = str(x)
        submissionString = re.search(regexExpression, tempSubStr).group(1)
        numberString = re.search(regexExpression, tempNumStr).group(1)
        submission_counter(submissionString, numberString)

    print(submission_dictionary)